In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings 
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
def wrangle(filepath):
    # read csv file
    df = pd.read_csv(filepath)
    # Encoding ordinal category value
    obesity_order = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }
    # Consumption of alcohol (CALC
    calc_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2    
    }

    #  Calories consumption monitoring (SCC)
    yes_no_order={'no':0,'yes':1}

    # Consumption of food between meals (CAEC)
    caec_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    transpot_order={
        'Walking':0,
        'Bike':1,
        'Motorbike':2,
        'Public_Transportation':3,
        'Automobile':4

    }
    
    gender_order={'Female':1,'Male':0}
    
    df['Gender']=df['Gender'].replace(gender_order)
    df['NObeyesdad']=df['NObeyesdad'].replace(obesity_order)
    df['CALC']=df['CALC'].replace(calc_order)
    df['SCC']=df['SCC'].replace(yes_no_order)
    df['SMOKE']=df['SMOKE'].replace(yes_no_order)
    df['family_history_with_overweight']=df['family_history_with_overweight'].replace(yes_no_order)
    df['FAVC']=df['FAVC'].replace(yes_no_order)
    df['CAEC']=df['CAEC'].replace(caec_order)
    df['MTRANS']=df['MTRANS'].replace(transpot_order)
    
    return df

In [3]:
# read csv file via wrangle function
df=wrangle("/kaggle/input/playground-series-s4e2/train.csv")
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,0,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,0,2.763573,0,0.000000,0.976473,1,3,3
1,1,1,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,0,2.000000,0,1.000000,1.000000,0,4,1
2,2,1,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,0,1.910378,0,0.866045,1.673584,0,3,0
3,3,1,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,0,1.674061,0,1.467863,0.780199,1,3,6
4,4,0,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,0,1.979848,0,1.967973,0.931721,1,3,3


In [4]:
target_column = 'NObeyesdad'
X = df.drop(['id','NObeyesdad'], axis=1)
y = df[target_column]

In [5]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [6]:
categorical_features = ['Gender','CALC','SCC','SMOKE','family_history_with_overweight','FAVC','CAEC','MTRANS']


In [7]:
params = {
    'objective': 'multiclass',
    'num_class': 7,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9
}



In [8]:
# Initialize variables to track the best model
best_model = None
best_accuracy = 0.0

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Create LightGBM datasets for training and validation
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features)

    # Train the LightGBM model
    lgb_model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, val_data])

    # Make predictions on the validation set
    y_pred_prob = lgb_model.predict(X_val)
    y_pred = [list(x).index(max(x)) for x in y_pred_prob]

    # Evaluate the model on the validation set
    accuracy = accuracy_score(y_val, y_pred)

    # Update best model if the current model is better
    if accuracy > best_accuracy:
        best_model = lgb_model
        best_accuracy = accuracy

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2050
[LightGBM] [Info] Number of data points in the train set: 16606, number of used features: 16
[LightGBM] [Info] Start training from score -2.107657
[LightGBM] [Info] Start training from score -1.907167
[LightGBM] [Info] Start training from score -2.146046
[LightGBM] [Info] Start training from score -2.108153
[LightGBM] [Info] Start training from score -1.964755
[LightGBM] [Info] Start training from score -1.855022
[LightGBM] [Info] Start training from score -1.635117
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2057
[LightGBM

In [9]:
best_accuracy

0.910645472061657

In [10]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    obesity_order = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }
    # Consumption of alcohol (CALC
    calc_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    #  Calories consumption monitoring (SCC)
    yes_no_order={'no':0,'yes':1}

    # Consumption of food between meals (CAEC)
    caec_order={
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    }

    transpot_order={
        'Walking':0,
        'Bike':1,
        'Motorbike':2,
        'Public_Transportation':3,
        'Automobile':4

    }
    
    gender_order={'Female':1,'Male':0}
    
    #drop id
    df=df.drop(['id'], axis=1)
    
    df['Gender']=df['Gender'].replace(gender_order)
    df['CALC']=df['CALC'].replace(calc_order)
    df['SCC']=df['SCC'].replace(yes_no_order)
    df['SMOKE']=df['SMOKE'].replace(yes_no_order)
    df['family_history_with_overweight']=df['family_history_with_overweight'].replace(yes_no_order)
    df['FAVC']=df['FAVC'].replace(yes_no_order)
    df['CAEC']=df['CAEC'].replace(caec_order)
    df['MTRANS']=df['MTRANS'].replace(transpot_order)
    
    #predicton
    y_pred_prob = best_model.predict(df)
    y_test_pred = [list(x).index(max(x)) for x in y_pred_prob]
    
    
    
    return y_test_pred

In [11]:
prediction=wrangle("/kaggle/input/playground-series-s4e2/test.csv")

In [12]:
len(prediction)

13840

In [13]:
test_df=pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
id=test_df['id']

In [14]:
submission_df = pd.DataFrame({'id': id, 'NObeyesdad': prediction})

In [15]:
obesity_order = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6
}

# convert numercial value into orginal ordinal value
submission_df['NObeyesdad'] = submission_df['NObeyesdad'].map({v: k for k, v in obesity_order.items()})

In [16]:
submission_df.set_index('id', inplace=True)

In [17]:
submission_df.to_csv('/kaggle/working/my_submissionrrr8.csv')